## Decision Tree Algorithm From Scratch
This short project was inspired by one of my projects at work. At some point this year, I found myself translating SAS script to Python script. One of my previous projects involved replicating a decision tree algorithm originally built in SAS Enterprise Miner into a Python script. So, in this project, I will be building a decision tree algorithm from scratch in Python, and to show this algorithm in action, I have applied it to a regression problem in this work book.

#### Import Libaries
Through out this work book, I have used just two popular libaries - numpy and pandas. 

In [56]:
import pandas as pd 
import numpy as np 

#### Dataset
To implement the decision tree algorithm, I have a real dataset of house prices sold in Seattle, Washington, USA., between August and December 2022. The goal is to predict the average housing price of a home given some relevant features of a home.

Relevant features considered for model building were: 

* beds: This is the number of bedrooms in a house 
* baths: This is the number of bathrooms in a house, where rows with 0.5 bath indicates a half bath, which translates to a bath with just the sink and water closet without a tub or shower.
* size: This is the total floor area of a house
* size_units: This is the unit of measurement of the total floor area of a house 
* lot_size: This is the total area of the land where a house is located. 
* lot_size_unit: This is the unit of measurement of the total land area 
* zip_code: This is postal address that pin-points a house
* price: This is the amount a sold was sold, and this is the variable of interest. 

Important Note: Some of the unit of measurements are either in sqft or acre. For uniformity, I have used sqft for all measurements.

In [57]:
training_dataset = "C:/Users/oyeni/Python Script/DecisionTreeFromScratch/Data/seattle_train.csv"
train = pd.read_csv(training_dataset, sep = ',')

In [58]:
testing_dataset = "C:/Users/oyeni/Python Script/DecisionTreeFromScratch/Data/seattle_test.csv"
test = pd.read_csv(testing_dataset, sep = ',')

#### Quick EDA 

In [59]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2016 entries, 0 to 2015
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   beds            2016 non-null   int64  
 1   baths           2016 non-null   float64
 2   size            2016 non-null   float64
 3   size_units      2016 non-null   object 
 4   lot_size        1669 non-null   float64
 5   lot_size_units  1669 non-null   object 
 6   zip_code        2016 non-null   int64  
 7   price           2016 non-null   float64
dtypes: float64(4), int64(2), object(2)
memory usage: 126.1+ KB


Here, I invoked info() method on the dataset. The method returns the features and the corresponding data types. Here, zipcode is read as integer. This is not right, albeit, it appears so by just looking at the zip code column. For the purpose of this model, I changed its data type before feeding the dataset to the algorithm.

In [60]:
train.isnull().sum()

beds                0
baths               0
size                0
size_units          0
lot_size          347
lot_size_units    347
zip_code            0
price               0
dtype: int64

In [61]:
test.isnull().sum()

beds               0
baths              0
size               0
size_units         0
lot_size          77
lot_size_units    77
zip_code           0
price              0
dtype: int64

Here, I invoked isnull().sum() on the dataset. This method returns the dataset features and the corresponding count of missing values. lot_size and lot_size_units had missing values on both train and test datasets. This also needs to be addressed before model is trained.

#### Handling Unit of Measurement

In [62]:
train['lot_size_units'].value_counts(dropna=False)

sqft    1449
NaN      347
acre     220
Name: lot_size_units, dtype: int64

In [63]:
test['lot_size_units'].value_counts(dropna=False)

sqft    369
NaN      77
acre     59
Name: lot_size_units, dtype: int64

""lot_size_units" has two different unit of measurements. When the data were collected, the data collector recorded lot size in acre and in square foot. This can be misleading to the algorithm. Applying value_counts() on the dataset with dropna=False option revealed the count of NAN a.k.a missing values, in lot_size_units.

For this analysis, I have used square foot as the unit of measurement. Hence, to convert lot size recorded in acre to square foot, I multiplied the "lot_size" by 43560, since 1 acre equates 43560 square feet.

In [64]:
train.loc[(train['lot_size_units'] == "acre"), 'lot_size'] = train['lot_size']*43560
train['lot_size_units'] = train['lot_size_units'].str.replace("acre", "sqft")

In [65]:
test.loc[(test['lot_size_units'] == "acre"), 'lot_size'] = test['lot_size']*43560
test['lot_size_units'] = test['lot_size_units'].str.replace("acre", "sqft")

#### Handling Missing data 

"lot_size" feature had 347 and 77 missing rows in both training and testing datasets, respectively. To handle this, I replaced the missing rows with the average of the entire column. There are other ways to handle missing data, but for the purpose of this project, I decided to impute missing values with mean.  

In [66]:
train['lot_size'].mean()

18789.951947273817

In [67]:
test['lot_size'].mean()

8961.000000000002

In [68]:
# replace missing rows in "lot_size" with means 
train.loc[(train['lot_size'].isnull()), 'lot_size'] = train['lot_size'].mean()

# replace missing rows in "lot_size_units" with "sqft"
train.loc[(train['lot_size_units'].isnull()), 'lot_size_units'] = 'sqft'

In [69]:
# replace missing rows in "lot_size" with means 
test.loc[(test['lot_size'].isnull()), 'lot_size'] = test['lot_size'].mean()

# replace missing rows in "lot_size_units" with "sqft"
test.loc[(test['lot_size_units'].isnull()), 'lot_size_units'] = 'sqft'

#### Change Zip Code Data Type 

In [70]:
train['zip_code'] = train['zip_code'].astype(str)
test['zip_code'] = test['zip_code'].astype(str)

In [71]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2016 entries, 0 to 2015
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   beds            2016 non-null   int64  
 1   baths           2016 non-null   float64
 2   size            2016 non-null   float64
 3   size_units      2016 non-null   object 
 4   lot_size        2016 non-null   float64
 5   lot_size_units  2016 non-null   object 
 6   zip_code        2016 non-null   object 
 7   price           2016 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 126.1+ KB


#### Train-Test

In [72]:
X_train = train.drop('price', axis = 1).values
Y_train = train.iloc[:, -1].values.reshape(-1,1)

X_test = test.drop('price', axis = 1).values
Y_test = test.iloc[:, -1].values.reshape(-1,1)

### Decision Tree Algorithm

There are two types of nodes in a decision tree, namely:

* Decision node: contains a condition to split the data
* Leaf node: contains the output variable which is used to make a prediction.

#### Node class

In [73]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left_child=None, right_child=None, var_red=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left_child = left_child
        self.right_child = right_child
        self.var_red = var_red
        
        # for leaf node
        self.value = value

#### Tree class

Here, I defined a class object called DecisionTreeRegressor(), just like in scikit-learn. This class contains methods that perform specific task, from tree building, splitting, to printing tree. 

In [74]:
class DecisionTreeRegressor():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # set tree foundation a.k.a root 
        self.root = None
        
        # set decision tree hyperparameters
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, df, curr_depth=0):
        ''' build tree function - this recursively builds the tree '''
        
        X, Y = df[:,:-1], df[:,-1]
        sample_size, feature_size = np.shape(X)
        best_split = {}
        # split until stopping conditions are met
        if sample_size>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(df, sample_size, feature_size)
            # check if variance reduction is positive
            if best_split["var_red"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["df_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["df_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["var_red"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, df, sample_size, feature_size):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_var_red = -float("inf")
        # loop over all the features
        for feature_index in range(feature_size):
            feature_values = df[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                df_left, df_right = self.split(df, feature_index, threshold)
                # check if childs are not null
                if len(df_left)>0 and len(df_right)>0:
                    y, left_y, right_y = df[:, -1], df_left[:, -1], df_right[:, -1]
                    # compute information gain >> variance reduction 
                    curr_var_red = self.variance_reduction(y, left_y, right_y)
                    # update the best split if needed
                    if curr_var_red>max_var_red:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["df_left"] = df_left
                        best_split["df_right"] = df_right
                        best_split["var_red"] = curr_var_red
                        max_var_red = curr_var_red
                        
        # return best split
        return best_split
    
    def split(self, df, feature_index, threshold):
        ''' function to split the data '''
        
        # splits data point to the left branch 
        df_left = np.array([row for row in df if row[feature_index]<=threshold])
        # splits data point to the right branch
        df_right = np.array([row for row in df if row[feature_index]>threshold])
        return df_left, df_right
    
    def variance_reduction(self, parent, l_child, r_child):
        ''' function to compute variance reduction -- information gain'''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        reduction = np.var(parent) - (weight_l * np.var(l_child) + weight_r * np.var(r_child))
        return reduction
    
    def calculate_leaf_value(self, Y):
        ''' function to compute predicted value at leaf node '''
        
        val = np.mean(Y)
        return val
                
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.var_red)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left_child, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right_child, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
        
    def make_prediction(self, x, tree):
        ''' function to predict new dataset '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left_child)
        else:
            return self.make_prediction(x, tree.right_child)
    
    def predict(self, X):
        ''' function to predict a single data point '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions

In [75]:
model = DecisionTreeRegressor(min_samples_split=2, max_depth=5)
model.fit(X_train, Y_train)
model.print_tree()

X_2 <= 2840.0 ? 110762108886.55444
 left:X_2 <= 1734.0 ? 41906843701.24878
  left:X_0 <= 1 ? 14730735665.033264
    left:X_2 <= 1250.0 ? 12954779282.883347
        left:X_2 <= 781.0 ? 4176807931.641815
                left:X_2 <= 460.0 ? 6095117392.498131
                                left:822000.0
                                right:401305.15873015876
                right:X_2 <= 945.0 ? 4404529021.937183
                                left:534457.1282051282
                                right:684989.2857142857
        right:X_4 <= 41817.6 ? 199666805555.55557
                left:X_6 <= 98105 ? 30450250000.0
                                left:1350000.0
                                right:913750.0
                right:2200000.0
    right:X_2 <= 730.0 ? 22497061648.7594
        left:X_2 <= 724.0 ? 13445435929390.162
                left:X_4 <= 18789.951947273817 ? 11815231666.765432
                                left:523464.0
                                right:998000.0

In [79]:
#Function to estimate root mean square error
def rmse(actual, pred, X_data):
    Y_pred = model.predict(X_data)
    MSE = np.square(np.subtract(actual,pred)).mean()
    RMSE = np.sqrt(MSE)
    return RMSE


In [77]:
#Prediction on validation dataset
rmse(Y_test, Y_pred, X_test)

827339.3610334604

In [78]:
#Prediction on training dataset
rmse(Y_train, Y_pred, X_train)

1098542.476033523

Voila, there you have it!! It is obvious the algorithm didn't generalize well to a new dataset, as the root mean square error (RMSE) on the test dataset is significantly high. The goal here is to optimize the loss function such that RMSE is close to zero. The RMSE on the training set is also high, which simply reveals that the algorithm is not anywhere near a production algorithm. There are couple of ways this algorithm can be improved, to mention a few:

* Increase the training dataset
* Add more relevant features 
* Apply the concept of hyperparameter tunning to choose the best hyperparameters for the algorithm